In [1]:
#import library
from bs4 import BeautifulSoup
import requests
import pandas as pd
from itertools import repeat

In [2]:
#get data from url
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(website_url,'lxml')

In [47]:
my_table = soup.find("table", { "class" : "wikitable sortable"})

In [5]:
def convert_to_list(bs4row):
    list_bs4row = bs4row.findAll(["td","th"])
    return [bs4.get_text().strip() for bs4 in list_bs4row]

In [6]:
#get rows
rows=my_table.findAll("tr")
#get header
header = convert_to_list(rows[0])
header

['Postcode', 'Borough', 'Neighbourhood']

In [7]:
my_data = [convert_to_list(r) for r in rows[1:]]

In [40]:
# All Data
df = pd.DataFrame(my_data,columns=header)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [41]:
# Except "Not assigned"
df = df[~df['Borough'].isin(['Not assigned'])]
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### Duplicate Data into one row

In [42]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()


In [44]:
df.shape

(103, 3)

In [45]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"
